In [140]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [141]:
# Get MNIST dataset
def get_mnist():
    from tensorflow.examples.tutorials.mnist import input_data
    mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
    
    x_train, y_train = mnist[0].next_batch(55000)
    x_valid, y_valid = mnist[1].next_batch(5000)
    x_test, y_test = mnist[2].next_batch(10000)
        
    return ([x_train, y_train], [x_valid, y_valid], [x_test, y_test])

In [142]:
def display_training_image(train_ix):
    x = x_train[train_ix]
    img = np.reshape(x, (28, 28))
    y = y_train[train_ix]

    plt.imshow(1-img, cmap=plt.get_cmap('gray'))
    plt.show()
    print np.where(y != 0)[0][0]

In [143]:
# Build graph
def build_network(n_inputs, n_outputs):
    x = tf.placeholder('float', [None, n_inputs], name='x')
    y = tf.placeholder('float', [None, n_outputs], name='y')

    w = tf.Variable(tf.random_uniform([n_inputs, n_outputs], name='rng'), name='w')
    b = tf.Variable(tf.zeros([n_outputs], name='zeros'), name='b')
    output = tf.add(tf.matmul(x, w, name='wx'), b, name='wx_plus_b')

    error = tf.nn.sigmoid_cross_entropy_with_logits(output, y, name='error')
    train_step = tf.train.GradientDescentOptimizer(0.001, name='grad_descent').minimize(error)
    
    return (x, y, w, b, train_step)

In [144]:
# Predict
def predict(x, w, b):
    y = np.dot(x, w) + b
    return np.argmax(y, axis=1)

In [145]:
# Run graph
[x_train, y_train], [x_valid, y_valid], [x_test, y_test] = get_mnist()
n_training, n_features = x_train.shape
n_classes = y_train.shape[1]

x, y, w, b, train_step = build_network(n_features, n_classes)

max_iter = 2000
batch_size = 100
with tf.Session() as session:
    session.run(tf.initialize_all_variables())
    x_train_batches = np.vsplit(x_train, n_training/batch_size)
    y_train_batches = np.vsplit(y_train, n_training/batch_size)
    for i in xrange(max_iter):
        for x_train_batch, y_train_batch in zip(x_train_batches, y_train_batches):
            session.run(train_step, feed_dict={x: x_train_batch, y: y_train_batch})
    weights = session.run(w)
    biases = session.run(b)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [146]:
y_train_predict = predict(x_train, weights, biases)
y_train_label = np.argmax(y_train, axis=1)
print np.sum(y_train_predict == y_train_label)/(len(y_train_label)+0.0)*100

92.9309090909


In [147]:
y_test_predict = predict(x_test, weights, biases)
y_test_label = np.argmax(y_test, axis=1)
print np.sum(y_test_predict == y_test_label)/(len(y_test_label)+0.0)*100

91.95
